In [2]:
%pip install langchain-community langchain-chroma

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/cc/19/f8af1cdefe326730ae02bd653f7382693153baf0bac7a69537d7811cad5f/langchain_community-0.3.7-py3-none-any.whl (2.4 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/bc/8c/ef0a589e523e72373a79a64cd1184682155113b865f889d815a9f264a755/langchain_chroma-0.1.4-py3-none-any.whl (10 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c3/be/d0d44e092656fe7a06b55e6103cbce807cdbdee17884a5367c68c9860853/dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e1/9b/a181f281f65d776426002f330c31849b86b31fc9d848db62e16f03ff739f/httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/49/09/72630413a7ded27684e33392a0ff52ff1c8ea6749fee641319e75f82072b/langchain-0.3.7-py3-none-any.whl (1.0 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/a1/25/5bd49cda589e98908e

In [3]:
%pip install -U langchain langchain-openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/56/f3/b8767fd4f33fb811139292943083f9bef0f8cc4a43edeff2c19b501df75f/langchain_openai-0.2.9-py3-none-any.whl (50 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/81/b2/9d8939c9ef73e6a2d5cb3366ef3fabd728a8de4729210d9af785c0edc6ec/openai-1.55.0-py3-none-any.whl (389 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.6/982.6 kB 759.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.4/304.4 kB 294.7 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install gpt4all

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/19/92/bad46a9bc5b993727045d09ce35e2dcc3939be02f47acfc99b2be0d33b33/gpt4all-2.8.2-py3-none-macosx_10_15_universal2.whl (6.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [6]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [7]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [8]:
import pandas as pd

df = pd.read_csv("o1_labels.csv")

texts = df['text_translated'].tolist()

In [14]:
! brew upgrade sqlite

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:720d4fb1164e600f787d656019a8e46314dc38e1885f4a8df809c180acf5e7b3
######################################################################### 100.0%
==> Pouring portable-ruby-3.3.6.el_capitan.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
ansible@10          dotnet@8            kanata              onnx
azqr                energy              kuzco               pytest
beautysh            eslint_d            libcutl             screenpipe
bibtex-tidy         fileql              libtatsu            sigsum-go
cbfmt               fltk@1.3            libxsd-frontend     tcl-tk@8
cljfmt              gci                 mergiraf            toml2json
cmake-lint   

In [15]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModel
# import torch
from langchain.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document


texts = " ".join(texts)
docs = [Document(page_content=text) for text in texts]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=7000, chunk_overlap=700)
splits = text_splitter.split_documents(docs)

print(len(splits))

# Create the vectorstore
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings()
)
# vectorstore = Chroma(
#     collection_name=splits,
#     embedding_function=embeddings  # Where to save data locally, remove if not necessary
# )
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is RTI"):
    print(chunk, end="", flush=True)

KeyboardInterrupt: 

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
from langchain.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document



docs = [Document(page_content=text) for text in texts]
print(len(docs))
print(docs[0])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=2000)
splits = text_splitter.split_documents(docs)
print(splits[:5])


# Create the vectorstore
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GPT4AllEmbeddings()
)
# vectorstore = Chroma(
#     collection_name=splits,
#     embedding_function=embeddings  # Where to save data locally, remove if not necessary
# )
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is RTI"):
    print(chunk, end="", flush=True)

313
page_content='"x" "1" "FTI - Strategy 2030   Federal government strategy for research, technology and innovation   FTI - Strategy 2030   Federal government strategy for research, technology and innovation   Vienna, 2020   imprint   Media owner, publisher and editor: Federal Government of the Republic of Austria   Vienna, 2020   contents   Introduction 4 The RTI location Austria 4 Perspectives of the RTI strategy 2030 5 What is the RTI strategy 2030 based on? 5 RTI Strategy 2030: Involvement of Stakeholders 6 RTI Strategy 2030: Pioneering Framework 6   Goals 6   Goal 1: catch up with the international top field and the FTI location   Strengthening Austria 7 Goal 2: Focus on effectiveness and excellence 7 Goal 3: Rely on knowledge, talents and skills 7 Central fields of action 8   Areas of action Objective 1: catch up with the top international field   and strengthen the RTI location Austria 8 fields of action objective 2: focus on effectiveness and excellence 9 fields of action obje